# Assignment Applied Machine Learning BSc IK 

## Notebook made by

**Gebruik graag dit formaat**

* Voor de namen:  voornaam rest van je naam, voornaam rest van je naam,....
* je studentnummers: hetzelfde: scheidt met `,`
* je emails: hetzelfde: scheidt met `,`
* voor je groep: **alleen de hoofdletter** (iets als  `A` of `B` dus)

__Namen__:Anoniem
__Emails__:Anoniem
__Student id__:Anoniem
__Groep__:Anoniem

## Toelichting

* Een aantal opgaven worden automatisch nagekeken. Bij vrijwel alle opdrachten staan er een paar tests onder de opdracht, dit is voornamelijk om te zorgen dat je de juiste type output geeft. Dit zijn dus *NIET* alle tests, die komen er bij het graden nog bij.
* Elke vraag is 1 punt waard, tenzij anders aangegeven. Soms is die punt onderverdeeld in deelpunten, maar niet altijd. 

## Voor het inleveren!

* Pas niet de cellen aan, vooral niet die je niet kunt editen. Dit levert problemen op bij nakijken. Twijfel je of je per ongeluk iets hebt gewijzigd, kopieer dan bij inleveren je antwoorden naar een nieuw bestand, zodat het niet fout kan gaan.

* Zorg dat de code goed runt van boven naar beneden, verifieer dat door boven in Kernel -> Restart & Run All uit te voeren

## Na het inleveren!

* Het gebeurt erg vaak dat mensen een "leeg bestand" inleveren. Vaak een andere versie van de opgave die nog ergens op je computer rondslingerde. Zonde van al je werk toch!
* Dus, lever **minstens een half uur voor tijd in**. Download dan wat je hebt ingeleverd op Canvas. Geef het een andere naam om verwarring te voorkomen. En draai alle cellen, en bekijk het. Geen syntax fouten? Alle vragen gemaakt? Dan zit het vast wel goed, en hoef je niet in de zenuwen te zitten.

# Applied Machine Learning W3

In the assignment for this week the focus will be on evaluation, where we take a look at confusion matrices, different metrics, and evaluation techniques such as cross validation and confidence intervals. As always, the exercises are mostly independent of each other, so if you get stuck somewhere, try doing a different part of the assignment, and come back to it later.

## Index

1. [Confusion Matrices](#conf_mat)
2. [Evaluation DIY](#eval1)
3. [Evluation with sklearn](#eval2)
4. [Cross Validation & Confidence Intervals](#cross_val)
5. [Baselines](#baselines)
6. [More Theory Questions](#theory)
7. [contagiousness Vs. Deadliness](#contag)

In [ ]:
import os
import sklearn
import numpy as np
import pandas as pd
import seaborn as sns

# testing
from nose.tools import assert_count_equal, assert_equal, assert_almost_equals
from numpy.testing import *
from numpy.testing import assert_equal
from pandas.testing import assert_frame_equal

# Please do not remove this: 
np.random.seed(31415)

<a id="conf_mat" />

# Confusion Matrix

In the class we looked at the following example Confusion Matrix with multiple classes:


* example  with 3 classes, N=1000
    * priors 700,200,100
    * diagonal : 600,50,10
    * predicted classes: 900,80,20
* Compute, precision, recall, F1 per class and  accuracy, macro values, weighted values over all classes.

## Your turn

* Create an example yourself, with maybe more classes and completely different numbers.
* Compute precision and recall per class alongside the table, indicate very very clearly what you are doing, and the order of things.
* Do it on paper with pencil and nice colors, make a photo, put it on the web, and create a hyperlink in the answer cell to your picture(s).

YOUR ANSWER HERE

<a id="eval1" />

# Evaluating a model

In this exercise we will be creating a complete ML pipeline where we load in a dataset, prepare it for classification by splitting it in train and test, run a classifier, and evaluate the results of this classifier.

We start by loading in the dataset, this code is already given for you.


In [ ]:
def loadfile():
    if 'faults.csv.gz' in os.listdir():
        return 'faults.csv.gz'
    elif os.path.exists('../../data/Week1/'):
        return '../../data/Week3/faults.csv.gz'
    elif os.path.exists('../../../data/Week1/'):
        return '../../../data/Week3/faults.csv.gz'
    
df= pd.read_csv(loadfile())
print(df.shape)
df.head()

The above dataframe contains data about the faults of machines. It is a multiclass classification problem with 6 fault classes and a 7th "other" class. It has 27 explanatory variables. Let's make a piechart of the data so that we can get a bit of a feel for the data.

In [ ]:
classes= ['Pastry', 'Z_Scratch', 'K_Scatch', 'Stains',
       'Dirtiness', 'Bumps', 'Other_Faults']
df[classes].sum().plot(kind='pie', title='Distribution of the classes among the 1941 instances.');
df[classes].sum().sort_values(ascending=False)

Next up is actually running the experiment, which we are going to to in the steps provided below.

### Start up
1. Add a column `target` to `df` which contains the label names from the `classes` as values and a row has label `X` as value in that column iff `df[X]==1`. (maybe there is a `numpy` function you can use?)
2. Drop the original columns of the labels from the dataframe. 
3. Do a stratified 70-30 data train test split. 
    * Indicate and check visually that the data is split stratified, so each class also has a 70-30 split.
4. Create `y_pred` using KNN. You may want to normalize the data first.

### Evaluate

1. Create the confusion table based on `y_pred` and `y_test` from scratch, that is not using any `sklearn` functions. 
    2. Also show it nicely using `sns.heatmap`. 
2. Compute precision, recall and F1 for each label.
3. Compute accuracy.
4. Compute Macro, micro and weighted versions of precision, recall and F1.
5. Create at least two non-learned rule based baseline predictions, compute evaluation metrics and put these together with the scores for your KNN model into a clear table.

### Starting up

In [ ]:
# start by adding the new column

# Fill in the right code in the line below
df['target'] = None
for x in range(df.shape[0]):
    for y in classes:
        if df.at[x,y] == 1:
            df.at[x,'target'] = y
new_df = df.drop(classes, axis=1)
# after adding the column, make matrices of the data for easy use

#WRITE YOUR CODE HERE
x = [x for x in range(len(classes))]
y = [y for y in classes]
z = (x,y)
print(z)
for a in range(len(new_df)):
    for b in z[0]:
        if new_df['target'][a] == z[1][b]:
            new_df['target'][a] = z[0][b]

faults_X, faults_y = np.array(new_df.iloc[:, :-1]), np.array(new_df['target'])

T=pd.Series(new_df.target).value_counts()
T.plot(kind='pie', title='Distribution of the classes among the 1941 instances.');
T

In [ ]:
assert_equal(new_df['target'].min(), 0)
assert_equal(new_df['target'].max(), 6)

Now that we have added the target column to the data we can create the train test split of the data, you may use `sklearn` for this.

In [ ]:
# start up split
from collections import Counter
from sklearn.model_selection import train_test_split
train_X, test_X, train_y, test_y = None, None, None, None

#WRITE YOUR CODE HERE
y = new_df['target']
y = np.array(y.astype('int'))
X = new_df[new_df.loc[:, new_df.columns != 'target'].columns]
train_X, test_X, train_y, test_y = train_test_split(X,y,test_size=0.3,stratify=y)

In [ ]:
# run the following print statements, if the proportions are similar
# it means that stratification is working correctly

print("Distribution of labels in the train set")
print(pd.Series(train_y).value_counts(normalize=True))
print()
print("Distribution of labels in the test set")
print(pd.Series(test_y).value_counts(normalize=True))


Next up is running the K Nearest Neighbours algorithm on the data, complete the function below, where you are allowed to use the sklearn function `KNeighborsClassifier`, we do this in a fucntion so that we can experiment with parameters later if we want to.

In [ ]:
# start up knn and compute the predictions
from sklearn.neighbors import KNeighborsClassifier

def knn_predict(train_features, train_labels, test_features, num_neighbors=5):
    y_pred = None
    classifier = KNeighborsClassifier(n_neighbors=num_neighbors)
    #WRITE YOUR CODE HERE
    classifier.fit(train_features, train_labels)
    y_pred = classifier.predict(test_features)
#     print(y_pred)
    return np.array(y_pred)

# DONT CHANGE THE LINE BELOW
y_pred = knn_predict(train_X, train_y, test_X, num_neighbors=5)

### Evaluation

Next up is creating the confusion matrix of the gold standard labels and our predictions. You are not allowed to use sklearn functions for this. Also, make sure that your code will work for any input of a gold standard array and a predicted array, regardless of the number of classes, so do not hardcode this!

**Hints**
We are following the definition of the confusion matrix where a row is the predicted class and the column is an actual class, like the first example in the HC slides with apples and oranges. (and the opposite of the wikipedia definition)

In [ ]:
# we expect the rows in the matrix to be predicted classes, and the columns to be the numbers for the gold
# standard classses, this is different from the sklearn representation!
from sklearn.metrics import confusion_matrix
def construct_confusion_matrix(gold_standard_labels, predicted_labels):
    # we will give you the first part, we create an empty square matrix
    # the size of the number of labels in the gold standard, understand why?
    number_of_classes = len(np.unique(gold_standard_labels))
    confusion_matrix = np.zeros((number_of_classes, number_of_classes))

    #WRITE YOUR CODE HERE
    for i in range(len(gold_standard_labels)):
#         confusion_matrix[gold_standard_labels[i]][predicted_labels[i]] += 1 #Sklearn & Wikipedia manier (actual = rij, 
                                                                                                       # pred. = kolom)
        confusion_matrix[predicted_labels[i]][gold_standard_labels[i]] += 1 #Hoorcollege manier (actual = kolom, pred = rij)

    return confusion_matrix
con_matrix = construct_confusion_matrix(test_y, y_pred)
con_matrix, #confusion_matrix(test_y, y_pred)

In the cell below make a nice confusion matrix here using `y_pred` and `sns.heatmap`

In [ ]:
#WRITE YOUR CODE HERE
sns.heatmap(construct_confusion_matrix(test_y, y_pred), annot=True)

### Calculating Accuracy, Precision Recall and F1 through the confusion matrix

Next up is calculating accuracy, precision, recall and F1. For this exercise you are not allowed to use the functions from sklearn, but you have to implement them yourself using the confusion matrix. Your task is to fill in the functions below, where the behaviour is similar to sklearn, where you can give in an 'average' parameter, which calculates the metrics with different weighting schemes.

**Hints**
- Try to think about precision and recall in terms of rows and columns in the confusion matrix, this should make the calculations relatively straightforward to implement in numpy.
- you can use `np.diag` to get the values on the diagonal of the confusion matrix.
- For F1 you can use the functions of precision and recall you wrote, but use the `None` option for weighting and calculate the different weights using these arrays, to avoid successive rounding errors in your results. Then use `np.nan_to_num` on the raw F1 array in case we have precision and recall both equal to 0.

In [ ]:
# complete the below functions, where you implement the following types for each metric:
# None: return the metric for each label separately, thus returning an array of size num_classes
# 'macro': return the macro weighted metric
# 'micro': returns the micro weighted metric
# 'weighted': returns the class weighted metric

# As a warmup, calculate accuracy over the entire test set
def accuracy(confusion_matrix): 
    accuracy = 0.0
    #WRITE YOUR CODE HERE
    accuracy = sum(np.diag(confusion_matrix)) / sum(sum(confusion_matrix))
    return np.round(accuracy,2)

def precision(confusion_matrix, average): 
    assert average in [None, 'macro', 'micro', 'weighted']
    #WRITE YOUR CODE HERE
    if average == None:
        row = np.array([sum(x) for x in confusion_matrix])
        precision = np.diag(confusion_matrix) / row
    if average == 'macro':
        row = np.array([sum(x) for x in confusion_matrix])
        prec = np.diag(confusion_matrix) / row
        precision = sum(prec) / len(confusion_matrix)
    if average == 'micro':
        precision = sum(np.diag(confusion_matrix)) / sum(sum(confusion_matrix))
    if average == 'weighted':
        row = np.array([sum(x) for x in confusion_matrix])
        prec = np.diag(confusion_matrix) / row
        distri = sum(confusion_matrix) / sum(sum(confusion_matrix))
        precision = sum(prec * distri)
    return np.round(precision,2)

def recall(confusion_matrix, average): 
    assert average in [None, 'macro', 'micro', 'weighted']
    #WRITE YOUR CODE HERE
    if average == None:
        recall = np.diag(confusion_matrix) / sum(confusion_matrix)
    if average == 'macro':
        rec = np.diag(confusion_matrix) / sum(confusion_matrix)
        recall = sum(rec) / len(confusion_matrix)
    if average == 'micro':
        recall = sum(np.diag(confusion_matrix)) / sum(sum(confusion_matrix))
    if average == 'weighted':
        rec = sum(np.diag(confusion_matrix)) / sum(sum(confusion_matrix))
        distri = sum(confusion_matrix) / sum(sum(confusion_matrix))
        recall = sum(rec * distri)
    return np.round(recall,2)
    
def f1(confusion_matrix, average):
    assert average in [None, 'macro', 'micro', 'weighted']
    F1 = 0.0
    #WRITE YOUR CODE HERE
    if average == None:
        row = np.array([sum(x) for x in confusion_matrix])
        F1 = np.diag(confusion_matrix)/(np.diag(confusion_matrix) + 0.5*(row + sum(confusion_matrix) - (np.diag(confusion_matrix)*2)))
#         F1 = 2*((recall(confusion_matrix,None)*precision(confusion_matrix,None))/(recall(confusion_matrix,None)+precision(confusion_matrix,None)))
    if average == 'macro':
        row = np.array([sum(x) for x in confusion_matrix])
        f1 = np.diag(confusion_matrix)/(np.diag(confusion_matrix) + 0.5*(row + sum(confusion_matrix) - (np.diag(confusion_matrix)*2)))
#         f1 = 2*((recall(confusion_matrix,None)*precision(confusion_matrix,None))/(recall(confusion_matrix,None)+precision(confusion_matrix,None)))
        F1 = sum(f1) / len(confusion_matrix)
    if average == 'micro':
        F1 = sum(np.diag(confusion_matrix)) / sum(sum(confusion_matrix))
    if average == 'weighted':
        row = np.array([sum(x) for x in confusion_matrix])
        f1 = np.diag(confusion_matrix)/(np.diag(confusion_matrix) + 0.5*(row + sum(confusion_matrix) - (np.diag(confusion_matrix)*2)))
#         f1 = 2*((recall(confusion_matrix,None)*precision(confusion_matrix,None))/(recall(confusion_matrix,None)+precision(confusion_matrix,None)))
        distri = sum(confusion_matrix) / sum(sum(confusion_matrix))
        F1 = sum(f1 * distri)
    return np.round(F1, 2)


In [ ]:
test_matrix = np.array([[1, 2, 0],
                       [0, 3, 1],
                       [0, 5, 2]])

assert_equal(type(precision(test_matrix, None)), np.ndarray)
assert_equal(type(recall(test_matrix, None)), np.ndarray)
assert_equal(type(f1(test_matrix, None)), np.ndarray)


In [ ]:
# Compute Macro, micro and weighted versions of precision, recall and F1.
# TODO: also use round to check here to make sure that we dont get small rounding differences
test_matrix = np.array([[1, 2, 0],
                       [0, 3, 1],
                       [0, 5, 2]])

assert_equal(type(precision(test_matrix, "macro")), np.float64)
assert_equal(type(recall(test_matrix, "macro")), np.float64)
assert_equal(type(f1(test_matrix, "macro")), np.float64)

assert_equal(type(precision(test_matrix, "micro")), np.float64)
assert_equal(type(recall(test_matrix, "micro")), np.float64)
assert_equal(type(f1(test_matrix, "micro")), np.float64)

assert_equal(type(precision(test_matrix, "weighted")), np.float64)
assert_equal(type(recall(test_matrix, "weighted")), np.float64)
assert_equal(type(f1(test_matrix, "weighted")), np.float64)


<a id="eval2" />

# Evaluate again

1.  Figure out how you can make a nice evaluation report using `sklearn`. Make such a report with `sklearn`.
2. Then make the same report using yoiur own functions. You need to add the support, and possibly more. Do that of course.

#tip: nice sklearn summary table that gives precision, recall, f1, support. Use that ;)
Also make the same report with your own functions, think about support as well

In [ ]:
#WRITE YOUR CODE HERE
from sklearn.metrics import classification_report
print(classification_report(test_y, y_pred))
recall(con_matrix, None)

<a id="cross_val" />

# Cross validation

* Do 5 fold cV for the KNN Classifier that we used in the previous exercise and the faults dataset, using the original `faults_X` and `faults_y arrays` we defined in the beginning.
* Does that make sense for the small classes? Think, experiment, look, and make a decision.
* Then run it. 
* Show for each class the distribution of the F1 values. Choose yourself how.
* Compute the confidence intervals. 
* What can you conclude?

For this exercise your are allowed to use functions from sklearn, and also use the `f1_score` from sklearn.

**Hints**
- If you calculate the F1 for each class individually, you can use a numpy array to store the results for each split, and then use numpy functions to easily calculate the confidence intervals.

Remember, we calculate confidence intervals for a class using $\mu \pm \frac{\sigma}{\sqrt{k}}$ where k is the number of folds.

In [ ]:
from sklearn.model_selection import KFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import f1_score, classification_report

# Don't forget about the `knn_predict` function you wrote!

#WRITE YOUR CODE HERE

<a id="baselines" />

# Baselines

Next we are going to implement some very simple baselines that do not use any information from the features, but instead rely only on the label. As with the previous functions, make sure that they work for any label column, so don't hardcode things! As these functions don't rely on the input features , the only input you get is the y label column, and a parameter that specifies the number of samples in the test set.

## Majority class

* Always choose the majority class

## Random 

* With $C$ classes, let each class have $\frac{1}{C}$ chance.

## Weighted Random 

*  Let each class have $P(C)$ chance.
    * $P(C)$ = fraction of C-instances in the population.
    
## Compute 

* Make a clear table with macro, micro, and weighted versions precision, recall and F1 for these three baselines for the `faults` dataset.  Clearly indicate how and why you decided to calculate things the way you did.

In [ ]:
# Majority class
from scipy.stats import mode # what does this function return, how can you use it?

def majority_class(input_labels, test_size):
    output_labels = []
    #WRITE YOUR CODE HERE
    return np.array(output_labels)


In [ ]:
assert_equal(len(majority_class([0, 1, 0, 1], test_size=20)), 20)

In [ ]:
from numpy.random import randint

def random(input_labels, test_size): 
    output_labels = []
    #WRITE YOUR CODE HERE
    return np.array(output_labels)

In [ ]:
assert_equal(len(random([0, 1, 0, 1], test_size=20)), 20)

In [ ]:
# Hint: have a look at the numpy.random.choice function with a distribution
# Another hint: you can get counts of unique values with np.unique (look at the documentation)
# which you can use to the probabilities for your distribution
from numpy.random import choice

def weighted_random(input_labels, test_size):
    output_labels = []
    #WRITE YOUR CODE HERE
    return np.array(output_labels)

In [ ]:
assert_equal(len(random([0, 1, 0, 1], test_size=20)), 20)

Now construct your table with results using the functions we just wrote above

In [ ]:
#WRITE YOUR CODE HERE

<a id ="theory" />

# Part 3: theory

The following questions are about precision, recall and accuracy.

Suppose you have a disease that occurs in 1 in a 100 people, and you have a test with an accuracy of .95. Meaning that in 95 percent of the case you correctly predict whether someone has the disease or not: The sum of the elements on the diagonal divided by the sum of all elements in the confusion matrix is .95.

1. What is de maximum (variable `maxR`) and minimal (`minR`) *recall* for the disease-class with these numbers?
2. What is de maximum (variable `maxP`) and minimal (`minP`) *precision* for the disease-class with these numbers?

**Hint** Construct a confusion matrix where you try different settings, make sure the numbers add up to the right totals.

## OLD DUTCH TEXT

Deze vragen gaan over precisie, recall en accuraatheid.

Stel je hebt een ziekte die bij 1 op de 100 mensen voorkomt. En je hebt een test met een accuraatheid van .95. Dit laatste betekent dat je in 95% van de gevallen juist voorspelt of iemand de ziekte heeft of niet. Anders gezegd: de som van de elementen op de diagonaal van de confusion matrix gedeeld door het totaal aantal elementen is .95.

1. Wat is de maximale(variabele `maxR`) en minimale (`minR`) *recall* voor de ziekte-klasse bij deze gegevens?
2. Wat is de maximale (`maxP`) en minimale (`minP`) *precision* voor de ziekte-klasse bij deze gegevens?

**Hint** Maak een confusion matrix waarin je verschillende settings uitprobeert. Zorg er steeds voor dat alles netjes optelt. 



In [ ]:
minP = None  # precisie
maxP = None
minR = None  # recall
maxR = None

#WRITE YOUR CODE HERE
# 1 op de 100 mensen -> 100 op de 10.000 mensen
# rijen -> predicted, kolommen -> actual
voorbeeld_matrix = np.array([[95, 495],
                             [5, 9405]])
precision(voorbeeld_matrix, None), recall(voorbeeld_matrix, None) 
# precision = 0.16 | recall = 0.95

In [ ]:
for m in {minP, maxP, minR, maxR}:
    assert 0 <= m <= 1

## Variant 

Dezelde vraag als boven, maar nu hebben 5 op de 100 mensen de ziekte en hebben we juist een accuraatheid van .99

In [ ]:
minP = None  # precisie
maxP = None
minR = None  # recall
maxR = None

#WRITE YOUR CODE HERE
# 5 op de 100 mensen -> 500 op de 10.000 mensen
# accuracy = .99
# rijen -> predicted, kolommen -> actual
voorbeeld_matrix2 = np.array([[495, 95],
                              [5, 9405]])
precision(voorbeeld_matrix2, None), recall(voorbeeld_matrix2, None)
# precision = 0.84 | recall = 0.99

In [ ]:
for m in {minP, maxP, minR, maxR}:
    assert 0 <= m <= 1

<a id="contag" />

# Part 4: What is worse: contagiousness of or deadliness?


Source: <https://www.nrc.nl/nieuws/2020/12/31/verdachte-virusvariant-met-opvallend-veel-mutaties-a4025804>

> What is worse: contagiousness or deadliness? At first glance you might think the second is. In the media there is thus also a sense of relief that the new variant of the virus is more more contagious, but less deadly than the existing variants. But is this justified? Adam Kucharski of the London School of Hygiene and Tropical Medicine [argued on Twitter](https://twitter.com/AdamJKucharski/status/1343567425107881986) that it is not.He gave the following numerical example

>Suppose R = 1.1. This means that on average ten people will infect eleven other people. Now suppose the generation time is 6 days i.e. it takes 6 days for those ten people to have infected these eleven people. This in 30 days this 'tree of contagion' splits 5 times, once each six days. Now suppose that of all people that become infected 0.8 percent die of the illness and the at the start we started with on day zero with 10.000 infected people. Then after one month we have $10.000 × (1,1)^5 × 0,8\% = 129$ people that have died.

>What happens when a mutation causes the virus to become 50 percent more deadly? If we will in the above formula with the changed deadliness we get the following: (1,1)^5 × (0,8\% × 1,5) = 193 $ people dying from the virus.

>Now what happens when the virus becomes 50 percent more contagious? Again, we can change the contagiousness in the formula and recalculate: $10.000 × (1,1 × 1,5)^5 × 0,8\% = 978$, meaning we now have almost a thousand people dying from the virus

>How is this possible? The deadliness of the virus is linearly correlated with the number of infrected people, while the number of infected people is related exponentially with the number of infected people, which we can also observe in the formula.

* Run the code below and try to understand what happens, do you understand why there is a difference in the number of dead in period 0, why it is the same for period 1, and why it starts to diverge after period 1?
* Check that the number of dead at the end of the month in the article is the same as the number at week 5 in the table.
* And see how we can very easily make a plot from spreadsheet data like this? We will be using this more in the coming weeks and explore even more cool ways of plotting data and results.

## Is it actually correct?
1. The formula is of course correct, however the outcome is not. Because if you have an R of 1.1 for a month you ahve had 6 'generations', and the total number of infected people is much higher than $10.000 x (1,1)^5$. Calculate how much more in the fuction `aantal_geinfecteerden_na_periodes`.

2. The same goes for the number of people that have died after a month, calculate this as well.



# OLD DUTCH VERSION
>Wat is erger: besmettelijkheid of dodelijkheid? Op het eerste gezicht denk je het tweede. In de media is dan ook opluchting te bespeuren dat het virus weliswaar besmettelijker is, maar geen ernstiger ziekteverloop veroorzaakt. Maar is dat terecht? Adam Kucharski van de London School of Hygiene and Tropical Medicine [betoogde op Twitter](https://twitter.com/AdamJKucharski/status/1343567425107881986) van niet. Hij gaf het volgende getallenvoorbeeld:

>Stel, R is 1,1. Dat wil zeggen dat tien besmette personen gemiddeld elf anderen besmetten. Stel, de generatietijd is zes dagen: het duurt zes dagen tot die tien mensen elf anderen hebben besmet, dus in 30 dagen vertakt die ‘besmettingsboom’ zich vijf keer. Stel, van alle besmette mensen overlijdt 0,8 procent. En stel, op tijdstip nul zijn 10.000 mensen besmet. Dan levert dat na een maand $10.000 × (1,1)^5 × 0,8\% = 129$ doden op.

>Wat gebeurt er dan als het virus door een mutatie 50 procent dodelijker wordt? Dan zijn na een maand $10.000 x (1,1)^5 × (0,8\% × 1,5) = 193 $
mensen overleden.

>Als het virus door een mutatie echter 50 procent besmettelijker wordt, dan ligt het dodental bijna vijf keer zo hoog: $10.000 × (1,1 × 1,5)^5 × 0,8\% = 978$ doden.


>Hoe dat kan? De dodelijkheid correleert lineair met het aantal doden, maar voor besmettelijkheid is dat verband exponentieel.

* Run onderstaande code. Snap wat er gebeurt. Snap je het verschil in aantal doden voor periode 0. En waarom is het gelijk in periode 1, en gaat het daarna (steeds sneller) uit elkaar lopen?
* Check dat het aantal doden aan het eind van de maand in het artikel hetzelfde is als in de tabel bij periode 5.
* En kijk eens hoe makkelijk je een plotje van zo'n spreadsheet maakt. Ideaal toch? Later in de cursus leren we hier meer over.

## Klopt het wel? (1pt)

1. De formule klopt natuurlijk, maar de uitkomst niet. Want als je een maand met een R van 1.1 zit heb je 6 "generaties", en dus in totaal veel meer besmette mensen gehad dan $10.000 x (1,1)^5$.  **Hoeveel meer dan?** **Bereken dat in de functie `aantal_geinfecteerden_na_periodes`.**
2. En dus hebben we na een maand ook een ander aantal overledenen.  Bereken dat eens.


In [ ]:
# zoals in het artikel
def aantaldoden(aantalbesmet=10**4,
                overlijdingsratio=.008,
                R=1.1,
                hoeveelerger=1.5,
                aantalperiodes=5):
    return {
        'dodelijker':
        round(aantalbesmet * (R)**aantalperiodes * overlijdingsratio *
              hoeveelerger),
        'besmettelijker':
        round(aantalbesmet * (R * hoeveelerger)**aantalperiodes *
              overlijdingsratio)
    }


# 50 % erger
A = {P: aantaldoden(hoeveelerger=1.5, aantalperiodes=P) for P in range(0, 6)}
A = pd.DataFrame(A).T
A.index.name = 'Periode'
A.plot(kind='bar', title='Aantal doden van geinfecteerden in periode x.')
A

In [ ]:
def aantal_geinfecteerden_na_periodes(
    aantalperiodes=6,
    aantalbesmet=10**4,
    R=1.1,
):
    '''Geef het totaal aantal geinfecteerde mensen terug na aantalperiodes generaties/periodes.
    Rond af op een geheel getal.'''
    #WRITE YOUR CODE HERE
    besmet = 0.0
    besmet += aantalbesmet * (R)**aantalperiodes
    return besmet

B = pd.Series({
    P: round(aantal_geinfecteerden_na_periodes(aantalperiodes=P))
    for P in range(21)
})
B.plot(kind='bar')
B

In [ ]:
assert isinstance(aantal_geinfecteerden_na_periodes(), int)



## Altered dataframe (1pt)

Example the code above again, where we construct teh dataframe `A`. Alter the code such that you print the correct numbers acorrding to the text above, and store this in dataframe `AA`. `AA` will have the same shape and index as `A`, just different numbers

* Also observe the difference between the titles of the two plots.


## Aangepast dataframe (1pt)

Bekijk de code hierboven nog eens waarin het Dataframe `A` gemaakt werd. Pas die code aan, zodat je daarmee het dataframe `AA` maakt dat de juiste aantallen print. `AA` heeft dezelfde *shape* als `A`, en ook dezelfde index en kolom namen. Alleen de waardes zijn anders. 

* Apprecieer ook het verschil in de titels tussen de twee plotjes.

In [ ]:
AA = A  # Change with your answer

#WRITE YOUR CODE HERE
AA = A.cumsum()
T = 'Totaal aantal doden na x periodes van 6 dagen.'
AA.plot(kind='bar', title=T)
print(AA)


#WRITE YOUR CODE HERE

In [ ]:
assert_equal(AA.shape, (6, 2))  # 2 kolommen, 6 rijen
assert_equal(list(AA.index) == list(range(6)),
             True)  # de index van AA is [0,1,2,3,4,5]

